# Import necessary libraries

In [3]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))
from db_connection.connection import PostgresConnection

# Add the src directory to the system path


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from customer_satisfaction_analysis import (
    calculate_scores, 
    calculate_satisfaction_score, 
    regression_model, 
    kmeans_clustering, 
    aggregate_clusters, 
    export_to_mysql
)